# KeyBERT로 topic 추출


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
# DataFrame 불러오기
user_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_user_df.csv')
biz_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_biz_df_filtered.csv')
review_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_review_df_filtered.csv')
active_area_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/active_area_df.csv')

<ipython-input-3-3a181239334e>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_user_df.csv')


In [4]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             int64  
 1   name                object 
 2   yelping_since       object 
 3   useful              int64  
 4   funny               int64  
 5   cool                int64  
 6   elite               object 
 7   friends             object 
 8   fans                int64  
 9   compliment_hot      int64  
 10  compliment_more     int64  
 11  compliment_profile  int64  
 12  compliment_cute     int64  
 13  compliment_list     int64  
 14  compliment_note     int64  
 15  compliment_plain    int64  
 16  compliment_cool     int64  
 17  compliment_funny    int64  
 18  compliment_writer   int64  
 19  compliment_photos   int64  
 20  review_count_user   float64
 21  average_stars_user  float64
dtypes: float64(2), int64(16), object(4)
memory usage: 333.7+ MB


In [5]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62586 entries, 0 to 62585
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   business_id        62586 non-null  int64  
 1   name               62586 non-null  object 
 2   address            61896 non-null  object 
 3   city               62586 non-null  object 
 4   state              62586 non-null  object 
 5   postal_code        62560 non-null  object 
 6   latitude           62586 non-null  float64
 7   longitude          62586 non-null  float64
 8   is_open            62586 non-null  int64  
 9   attributes         0 non-null      float64
 10  category           62586 non-null  object 
 11  region             62586 non-null  object 
 12  review_count_biz   62586 non-null  float64
 13  average_stars_biz  53450 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 6.7+ MB


In [6]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680161 entries, 0 to 2680160
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    int64  
 1   user_id      int64  
 2   business_id  int64  
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
 9   text_length  int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 204.5+ MB


In [7]:
active_area_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974954 entries, 0 to 974953
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               974954 non-null  int64 
 1   most_visited_regions  974954 non-null  object
 2   visit_count           974954 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.3+ MB


## 1. install library

In [8]:
!pip install keybert

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [9]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# 문장으로 분리 토크나이저 정의
tokenizer = PunktSentenceTokenizer()


## 2. 문장분리


In [1]:
doc=review_df.loc[2]['text']

NameError: name 'review_df' is not defined

In [44]:
sentences = tokenizer.tokenize(doc)
sentences

['I was really between 3 and 4 stars for this one.',
 'I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.',
 'The vibe is totally different as this is geared more to take out although they do have a decent size dining area.',
 'You order at the counter and they deliver it.',
 'My daughter and I tried the sushi bowl.',
 'You pick up a piece of paper and  select from a choice of proteins, vegetables, rice and toppings.',
 "I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want.",
 'The sushi bowl was good but not wonderful.',
 'The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items.',
 'The sauces were tasty.',
 'My husband ordered the Maui roll which is my favorite roll at the 96th street location and this just was not up that standard.',
 'It was good but the quality and consi

## 3. 토픽 추출

In [27]:
from keybert import KeyBERT

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(sentences)

In [47]:
# 각 문장에 대해 가장 연관 있는 키워드 1개 추출
keywords_per_sentence = []
for sentence in sentences:
    keywords = kw_model.extract_keywords(sentence, keyphrase_ngram_range=(1,1), stop_words=None, top_n=1)
    keyword = keywords[0][0] if keywords else ''
    keywords_per_sentence.append((sentence, keyword))

# 데이터프레임으로 변환
df_keywords = pd.DataFrame(keywords_per_sentence, columns=['Sentence', 'Keyword'])

# 출력 설정 변경
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

# 결과 출력
df_keywords

,Sentence,Keyword
0,I was really between 3 and 4 stars for this one.,stars
1,I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.,tchopstix
2,The vibe is totally different as this is geared more to take out although they do have a decent size dining area.,dining
3,You order at the counter and they deliver it.,order
4,My daughter and I tried the sushi bowl.,sushi
5,"You pick up a piece of paper and select from a choice of proteins, vegetables, rice and toppings.",vegetables
6,I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want.,server
7,The sushi bowl was good but not wonderful.,sushi
8,The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items.,rice
9,The sauces were tasty.,sauces


In [48]:
df_keywords.to_csv('df_keywords.csv', index=False)

In [41]:
kw_model.extract_keywords(sentences, top_n=(1,3), keyphrase_ngram_range=2,  stop_words=None)

[]

In [16]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None)

[('sushi bowl', 0.6826),
 ('sushi bowls', 0.6601),
 ('sushi place', 0.6006),
 ('sushi', 0.5825),
 ('the sushi', 0.5778)]

In [17]:
stopwords = [] # 불용

In [18]:
# 키워드 추출
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words=stopwords)

[('sushi bowl', 0.6826),
 ('the sushi bowl', 0.6815),
 ('the sushi bowls', 0.6666),
 ('sushi bowls', 0.6601),
 ('sushi bowl you', 0.62)]